## Simple Recommender

In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings_file = '/Users/gregoryolson/Documents/Data Science CT/Capstone/Capstone_Books/Data/ratings_cleaned.csv'
books_file = '/Users/gregoryolson/Documents/Data Science CT/Capstone/Capstone_Books/Data/books_cleaned.csv'

ratings = pd.read_csv(ratings_file)
books = pd.read_csv(books_file)

In [3]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [4]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
#ratings = ratings.rename(columns={'book_id': 'id'})
#df = pd.merge(ratings, books, on='id')

In [6]:
#df.head()

In [7]:
#df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

In [8]:
#df.groupby('title')['rating'].count().sort_values(ascending=False).head()

In [9]:
#ratings_mean_count = pd.DataFrame(df.groupby('title')['rating'].mean())

In [10]:
#ratings_mean_count['rating_counts'] = pd.DataFrame(df.groupby('title')['rating'].count())

In [11]:
#ratings_mean_count.head()

In [12]:
#user_movie_rating = df.pivot_table(index='user_id', columns='title', values='rating')

In [13]:
#user_movie_rating.head()

In [14]:
#nef_ratings = user_movie_rating['1984']

In [15]:
#nef_ratings.head()

In [16]:
#movies_like_nef = user_movie_rating.corrwith(nef_ratings)

#corr_nef = pd.DataFrame(movies_like_nef, columns=['Correlation'])
#corr_nef.dropna(inplace=True)
#corr_nef.head()

In [17]:
#corr_nef.sort_values('Correlation', ascending=False).head(10)

In [18]:
#corr_nef = corr_nef.join(ratings_mean_count['rating_counts'])
#corr_nef.head()

In [19]:
#corr_nef[corr_nef['rating_counts'] > 90].sort_values('Correlation', ascending=False).head()

## Surprise

In [20]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

In [21]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

In [22]:
benchmark = []
results = cross_validate(BaselineOnly(), data, measures=['RMSE'], cv=3, verbose=False)
tmp = pd.DataFrame.from_dict(results).mean(axis=0)
tmp = tmp.append(pd.Series([str(BaselineOnly()).split(' ')[0].split('.')[-1]], index=['Algorithm']))
benchmark.append(tmp)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [23]:
#benchmark = []
# Iterate over all algorithms
#for algorithm in [SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    #SVD(), SVDpp(), 
    # Perform cross validation
    #results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    #tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    #tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    #benchmark.append(tmp)

In [24]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.851301,3.644294,2.764772


In [25]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.8493222 , 0.84696754, 0.84751812]),
 'fit_time': (1.8949179649353027, 2.192225933074951, 2.2691309452056885),
 'test_time': (2.8051600456237793, 2.7943060398101807, 2.964186191558838)}

In [26]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.8410


0.8409544621127243

In [27]:
trainset = algo.trainset
print(algo.__class__.__name__)

BaselineOnly


In [28]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
df.head()

,uid,iid,rui,est,details,Iu,Ui,err
0,27124,7979,4.0,4.005939,{'was_impossible': False},1,76,0.005939
1,15232,8102,5.0,4.622632,{'was_impossible': False},23,76,0.377368
2,8914,3577,5.0,3.919125,{'was_impossible': False},59,72,1.080875
3,47490,673,4.0,4.490531,{'was_impossible': False},95,73,0.490531
4,10473,3989,5.0,4.020761,{'was_impossible': False},24,73,0.979239
